In [ ]:
# Expiry, symbol and option info extracted from NSE
import requests
import lxml.html as lh
import datetime as dt
import pandas as pd
import numpy as np
import re

symbol = 'ACC'
expiry = ''

url_base = "https://www.nseindia.com/marketinfo/companyTracker/mtOptionKeys.jsp?companySymbol="
url_end = "&indexSymbol=NIFTY&series=EQ&instrument=OPTSTK&date="
url = url_base+symbol+url_end+expiry
page = requests.get(url)
doc = lh.fromstring(page.content)

the_path= "//form"   # Get the form content. There are two forms
expiries = doc.xpath(the_path)[0].text_content() # 0 has the expiry date

strip_chars = [ord('\n'), ord('\xa0'), ord('\t')] # characters to be stripped
char_table = {s: ' ' for s in strip_chars} # table for translate to locate the chars

sym_exp = expiries.translate(char_table).split()
# symbol = sym_exp[2].strip('()') # Remove round () from the 3rd element for symbol

# Get the expiries only, from the table
expiry = [sym_exp[k+i] 
 for k, v in enumerate(sym_exp) 
 if v == '--Select--' 
 for i in range(len(sym_exp) - k)][1:]

# Convert to datetime
expiry_dt = [dt.datetime.strptime(date, "%d%b%Y").date() for date in expiry]

tbl_path = "//tr"
tr_elements = doc.xpath(tbl_path)

# Option table is contained in rows of length 21
opt_table = [t for t in tr_elements if len(t) == 21]
# tbl_head1 = [i.text_content() for i in opt_table[0]] # First row is header

# Recreate the header
tbl_head = ['cQuote', 'cOI', 'cChgOI', 'cLTP', 'cNetChg', 'cVolume', 
             'cBidQty', 'cBidPrice', 'cOfferPrice', 'cOfferQty', 'Strike', 
             'pBidQty', 'pBidPrice', 'pOfferPrice', 'pOfferQty', 
             'pVolume', 'pNetChg', 'pLTP', 'pOI', 'pChgOI', 'pQuote']

# Extract the opt_table
opt_list = [[n.text_content().strip() 
             for m, n in enumerate(v)] 
             for k, v in enumerate(opt_table[1:])]

# Create the dataframe with Strike and Expiry
df = pd.DataFrame(columns=tbl_head, data=opt_list).drop(['cQuote', 'pQuote'], 1)

df = df.replace(',', '', regex=True) # Remove comma from numbers
df = df.apply(pd.to_numeric, errors='coerce') # Convert to numeric
df.insert(0, 'Expiry', expiry_dt[0])
df.insert(0, 'Symbol', symbol)

# Rearrange columns
cols_beginning = ['Symbol', 'Expiry', 'Strike']
df[cols_beginning + [c for c in df if c not in cols_beginning]]
df